## Read in Data

In [1]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns

import io
from io import BytesIO
from urllib.parse import urlparse
import boto3

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import ImageFile, Image
import warnings

2024-08-03 19:49:42.408235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-03 19:49:42.408273: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-03 19:49:43.268929: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-03 19:49:43.269028: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-08-03 19:49:43.269037: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
warnings.simplefilter('ignore', Image.DecompressionBombWarning)
Image.MAX_IMAGE_PIXELS = None

In [4]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        

In [5]:
# Initialize S3 client
s3_bucket = '207-images-labels'
session = boto3.Session(
     aws_access_key_id= aws_access_key_id,
     aws_secret_access_key= aws_secret_access_key)
client = session.client('s3')

def to_s3_npy(data: np.array, s3_uri: str):
    bytes_ = BytesIO()
    np.save(bytes_, data, allow_pickle=True)
    bytes_.seek(0)
    parsed_s3 = urlparse(s3_uri)
    client.upload_fileobj(
        Fileobj=bytes_, Bucket=parsed_s3.netloc, Key=parsed_s3.path[1:]
    )
    return True

In [6]:
def load_data(path_to_data, subfolder):
    '''Load 2D images and their corresponding labels
    Parameters:
    path_to_data (str): This is the path to data
    
    Returns:
    images (np.ndarray): A numpy array of shape (N, 1000, 1000, 3)
    labels (np.ndarray): A numpy array of shape (N)
    
    '''
    bucket_name = '207-images-labels'

    ## load images and labels
    images = []
    labels = []
    file_names = []
    label_map = {'fake': 0, 'real': 1, 'synthetic': 2}

    
    for label in os.listdir(path_to_data):
        label_path = os.path.join(path_to_data, label)
        if os.path.isdir(label_path):
            for file_name in os.listdir(label_path):
                if file_name.endswith(".jpg") or file_name.endswith(".png"):
                    # Exclude images with label 'synthetic'
                    if label_map[label] == 2:
                        continue
                    file_names.append(os.path.join(label_path, file_name))
                                      
    k = 0   
    m = 0
    imgs = []
    img_arr = []
    for i, j in enumerate(file_names):
        if (i % 100 == 99):
            imgs.append(img_arr)
            img_arr = []
        img_arr.append(j)
        
    for i in imgs:
        for j in i:
            img = load_img(j, target_size=(1000, 1000))
            img_arr = img_to_array(img)
            images.append(img_arr)
            if 'real' in i:
                labels.append('real')
            if 'fake' in i:
                labels.append('fake')
            k += 1

            if (k < 1000):
                break
        images = np.array(images)
        labels = np.array(labels)

        to_s3_npy(images, f's3://207-images-labels/{subfolder}/images/img_numpy_{m}')
        to_s3_npy(labels, f's3://207-images-labels/{subfolder}/labels/label_numpy_{m}')
        
        m += 1
        images = []
        labels = []
            
    print("done")

In [7]:
def write_data_to_s3(bucket, key, data):
    np_bytes = io.BytesIO()
    np.save(np_bytes, data)
    np_bytes.seek(0)
    client.put_object(Bucket=bucket, Key=key, Body=np_bytes)

In [8]:
def augment_data(X, y):
    """
    Apply transformations and augmentations to data, and use black and white versions as additional features.
    Parameters:
    X  (np.ndarray)
    y  (np.ndarray)
    Returns:
    X_augmented (np.ndarray)
    y_augmented (np.ndarray)
    """
    # Convert images to black and white
    image_bw = tf.image.rgb_to_grayscale(X)
    image_bw = tf.image.grayscale_to_rgb(image_bw)
    # Randomly flip the images
    X_flipped = tf.image.random_flip_left_right(X)
    image_bw_flipped = tf.image.random_flip_left_right(image_bw)
    # Adjust brightness
    X_brightness = tf.image.random_brightness(X_flipped, max_delta=0.2)
    image_bw_brightness = tf.image.random_brightness(image_bw_flipped, max_delta=0.2)
    # Adjust contrast
    X_contrast = tf.image.random_contrast(X_brightness, lower=0.8, upper=1.2)
    image_bw_contrast = tf.image.random_contrast(image_bw_brightness, lower=0.8, upper=1.2)
    # Rotate the images
    X_rotated = tf.image.rot90(X_contrast, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image_bw_rotated = tf.image.rot90(image_bw_contrast, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    # Concatenate original and black and white images along the channel axis
    X_combined = tf.concat([X_rotated, image_bw_rotated], axis=0)
    y_combined = np.concatenate((y, y))
    # Rescale images by dividing each pixel by 255.0
    X_combined = X_combined / 255.0
    return X_combined.numpy(), y_combined

In [9]:
train_path = './data/raw/train'
test_path = './data/raw/test'
abs_train_path = os.path.abspath(train_path)
abs_test_path = os.path.abspath(test_path)

def load_and_process_batch(file_names, batch_size=100):
    images = []
    labels = []
    for file_name in file_names:
        img = load_img(file_name, target_size=(1000, 1000))
        img_arr = img_to_array(img)
        images.append(img_arr)
        if 'real' in file_name:
            labels.append(1)
        elif 'fake' in file_name:
            labels.append(0)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [10]:
def save_split_data(X, y, output_dirs, batch_index, prefix):
    num_batches = 1
    def save_data(X_split, y_split, split_name):
        num_batches = (len(X_split) + batch_size - 1) // batch_size

        start_idx = 0

        end_idx = -1
        save_data_batch(X_split, y_split,
                        output_dirs[split_name],
                        f'{prefix}_{split_name}_batch_{batch_index}')
        try:
            save_data(X, y, 'train')
        except:
            print("unable to save")
                
def save_data_batch(X, y, folder_name, filename_prefix):
    os.makedirs(folder_name, exist_ok=True)
    print("writing to ", folder_name)
    write_data_to_s3(s3_bucket, os.path.join(folder_name, f'images/{filename_prefix}_imgs.npy'), X)
    write_data_to_s3(s3_bucket, os.path.join(folder_name, f'labels/{filename_prefix}_lbls.npy'), y)
    print("finished writing batch")
    
    
def process_and_save_data(file_names, batch_size, output_dirs, split_ratios):
    for i in range(0, len(file_names), batch_size):
        batch_file_names = file_names[i:i + batch_size]
        images, labels = load_and_process_batch(batch_file_names, batch_size)
        images, labels = augment_data(images, labels)
        save_split_data(images, labels, output_dirs, i // batch_size + 1, 'img_numpy')

In [27]:
path_to_data = abs_train_path
batch_size = 10
split_ratios = (0.75, 0.05, 0.2)  # Train, validation, test
output_dirs = {
    'train': 'train/pre1',
    'val': 'data/pre1/val',
    'test': 'data/pre1/test'
}


file_names = []
label_map = {'fake': 0, 'real': 1, 'synthetic': 2}
for label in os.listdir(path_to_data):
    label_path = os.path.join(path_to_data, label)
    if os.path.isdir(label_path):
        for file_name in os.listdir(label_path):
            if file_name.endswith(".jpg") or file_name.endswith(".png"):
                if label_map[label] == 2:
                    continue
                file_names.append(os.path.join(label_path, file_name))
# Generate a permutation of indices
indices = list(range(len( file_names)))
random.shuffle(indices)
# Shuffle both lists using the same permutation
shuffled_list1 = [file_names[i] for i in indices]
process_and_save_data(shuffled_list1, batch_size, output_dirs, split_ratios)

writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished wr

writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished wr

finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to 

writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished wr

finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to 

...

writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished wr

finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to 

writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished wr

finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to 

writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished writing batch
writing to  train/pre1
finished wr

In [14]:
load_data(abs_train_path, 'train')
load_data(abs_test_path, 'test')

done
done
